# 모델 예측

* open-solar-ko를 가지고 주어진 이전 대화, 사용자 페르소나, 챗봇 페르소나로부터 bot의 다음 발화를 생성

## 평가 데이터 불러오기

src_list, context_list만 있는 dataframe으로 편집

In [ ]:
from tqdm import tqdm
import pandas as pd
eval_vl2 = pd.read_csv('korean_msc/preprocessing/VL_session2_geval_v6_sampled_banmal.csv')
# eval_vl3 = pd.read_csv('korean_msc/preprocessing/VL_session3_geval_v4_sampled.csv')
# eval_vl4 = pd.read_csv('korean_msc/preprocessing/VL_session4_geval_banmal_persona_pred.csv')

In [ ]:
eval_vl2.shape

In [ ]:
eval_vl2.head()

In [ ]:
eval_vl2['bot_persona'] = eval_vl2['bot_persona'].apply(lambda x: eval(x))
eval_vl2['user_persona'] = eval_vl2['user_persona'].apply(lambda x: eval(x))
eval_vl2['bot_dialog'] = eval_vl2['bot_dialog'].apply(lambda x: eval(x))
eval_vl2['user_dialog'] = eval_vl2['user_dialog'].apply(lambda x: eval(x))
eval_vl2['prev_dialog'] = eval_vl2.apply(lambda x: eval(x['prev_dialog'])[1:] + x['new_input'], axis=1)

In [ ]:
eval_vl2.head()

In [ ]:
from tqdm import tqdm
import pandas as pd

def preprocess(data):
    eval_df = pd.DataFrame(columns=['user_persona', 'bot_persona', 'prev_dialog', 'response'])
    
    tqdm.pandas()
    # '\n' join - row마다!
    eval_df['user_persona'] = data['user_persona'].progress_apply(lambda x : ''.join(x))
    eval_df['bot_persona'] = data['bot_persona'].progress_apply(lambda x : ''.join(x))
    eval_df['prev_dialog'] = data['prev_dialog'].progress_apply(lambda x : ''.join(x))
    eval_df['response'] = data['response'].progress_apply(lambda x : ''.join(x))
    
    return eval_df

In [ ]:
eval_vl2 = preprocess(eval_vl2)
eval_vl2.head()

In [ ]:
eval_vl2.loc[0, 'user_persona']

## Model Prediction


https://python.langchain.com/docs/integrations/llms/llamacpp

In [1]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_community.llms import LlamaCpp

n_gpu_layers = -1  # The number of layers to put on the GPU. The rest will be on the CPU. If you don't know how many layers there are, you can use -1 to move all to GPU.
n_batch = 4096  # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.

# Callbacks support token-wise streaming
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])


2가지로 구성

1. 최근 5-turn의 대화
2. 추출한 사용자 페르소나 + 최근 5-turn의 대화

-> 근데 이게 챗봇 페르소나가 아니라 MSC의 사용자 1과 이뤄진 대화이기 때문에 챗봇 페르소나를 넣는 것이 불가능!


----

따라서 챗봇 페르소나를 제외하고 실험 진행


> 프롬프트 수정 필요

In [3]:
_DEFAULT_TEMPLATE =  """지금부터 아래의 [챗봇 페르소나]를 가진 인물이 되어, [상대 페르소나]를 고려한 대화를 수행한다.
[챗봇 페르소나]
- 나이:{bot_age}
- 성별:{bot_gender}
- 성격:{bot_persona}

[사용자 기본 정보]
- 나이:{user_age}
- 성별:{user_gender}

[대화 예시]
Bot:나는 학원 수학강사로 일하고 있는데, 다음주부터 학생들이 기말고사라서 바빠져서 첼로연주를 쉬어야 해.
User:학원 수학 선생님이셨구나. 나도 일렉기타를 배우고 싶은데, 밴드부 동아리에는 관심이 없고 학원을 알아보려고 하는데 학원비용이 많이 비쌀까?
Bot:그렇게 비싸진 않을거야. 20만원 안에서 해결할 수 있을거야.
User:그정도면 아르바이트해서 모을 수 있는 금액이네. 첼로 연주 배운지는 얼마나 됐어?
Bot:3개월째 배우고 있는데, 중급단계를 배우고 있어.

[현재 발화 관련 사용자 정보]
{user_persona}

지금까지의 프롬프트를 읽고 [챗봇 정보]의 인물이 되어 대답하고, [사용자 정보]와 [현재 발화 관련 사용자 정보]에 기반하여 친절하고 예의있게 답변하라.
{prev_dialog}
User:{new_input}
Bot:"""

prompt = PromptTemplate.from_template(_DEFAULT_TEMPLATE)
prompt

PromptTemplate(input_variables=['bot_age', 'bot_gender', 'bot_persona', 'new_input', 'prev_dialog', 'user_age', 'user_gender', 'user_persona'], template='지금부터 아래의 [챗봇 페르소나]를 가진 인물이 되어, [상대 페르소나]를 고려한 대화를 수행한다.\n[챗봇 페르소나]\n- 나이:{bot_age}\n- 성별:{bot_gender}\n- 성격:{bot_persona}\n\n[사용자 기본 정보]\n- 나이:{user_age}\n- 성별:{user_gender}\n\n[대화 예시]\nBot:나는 학원 수학강사로 일하고 있는데, 다음주부터 학생들이 기말고사라서 바빠져서 첼로연주를 쉬어야 해.\nUser:학원 수학 선생님이셨구나. 나도 일렉기타를 배우고 싶은데, 밴드부 동아리에는 관심이 없고 학원을 알아보려고 하는데 학원비용이 많이 비쌀까?\nBot:그렇게 비싸진 않을거야. 20만원 안에서 해결할 수 있을거야.\nUser:그정도면 아르바이트해서 모을 수 있는 금액이네. 첼로 연주 배운지는 얼마나 됐어?\nBot:3개월째 배우고 있는데, 중급단계를 배우고 있어.\n\n[현재 발화 관련 사용자 정보]\n{user_persona}\n\n지금까지의 프롬프트를 읽고 [챗봇 정보]의 인물이 되어 대답하고, [사용자 정보]와 [현재 발화 관련 사용자 정보]에 기반하여 친절하고 예의있게 답변하라.\n{prev_dialog}\nUser:{new_input}\nBot:')

In [4]:
import pandas as pd

eval_vl2 = pd.read_csv('./korean_msc/preprocessing/VL_session2_geval_v6_sampled_banmal.csv')
eval_vl2.head()

,id,topic,num_sessions,bot_persona,bot_age,bot_gender,bot_occupation,user_persona,user_age,user_gender,user_occupation,bot_dialog,user_dialog,prev_dialog,new_input
0,K2-33237-CL21405-CP22628-07-07-S2.json,가족,2,"['나는 여자이다.', '나는 20대이다.', '나는 고양이 두 마리를 키운다.',...",20대,여성,사무직,"['나는 고양이 두 마리를 중성화 수술시키고 집으로 가는 길이다', '나는 친구랑 ...",20대,여성,기능직,"['경기도에 살고 있구나. 난 20대이고 같은 경기도에 사는 여자야.', '고양이를...","['안녕, 난 경기도에서 살고 있어. 20대 여자야.', '같은 도시 사람이네. 나...","['Bot:나는 학원 수학강사로 일하고 있는데, 다음주부터 학생들이 기말고사라서 바...",빠르게 습득한 거 아니야? 나는 고양이 케어하러 가야되서 다음에 연락할게.
1,K5-03865-CL74427-CP10762-07-02-S2.json,가족,2,"['나는 남자이다.', '나는 20대이다.', '나의 거주지는 대구이다.', '나는...",20대,남성,학생,['나의 형은 자동차 정비원이다'],20대,남성,무직,"['안녕, 나는 20대 남성이야.', '안녕. 10대면 지금 고등학생이겠네.', '...","['안녕, 나는 10대 남자야.', '응, 작곡가가 되기 위해서 음악 공부를 열심히...",['Bot:에이 그정도면 그래도 괜찮지. 우리 어머니는 남자 키가 175이상만 되면...,괜찮아. 너는 어머니하고 사이 좋아?
2,K2-08382-CL20425-CP21061-17-08-S2.json,건강,2,"['나는 남자이다.', '나는 20대이다.', '나는 커피를 최근에 끊었다.', '...",20대,남성,학생,['나는 미국으로 어학연수를 간다'],20대,남성,무직,"['안녕, 난 20대 남자야. 반가워.', '오늘 기분이 좋더라. 너는 무슨 일을 ...","['기운이 엄청 넘치네. 나도 20대 남자야. 반가워.', '나는 싫어하는 EDM를...","['Bot:깁스 몇 주만 하면 되는 건데, 괜찮아.', 'User:축구하다가 너무 ...","맞아, 그래서 챙길 수 있는 약들은 많이 챙겨가려고 해."
3,K2-31842-CL23868-CP23582-17-06-S2.json,건강,2,"['나는 여자이다.', '나는 20대이다.', '나는 요즘, 가끔 죄책감이 든다.'...",20대,남성,학생,"['나는 대학생이다', '나는 심리학이 부전공이다', '나는 카푸치노를 매일 2잔 ...",20대,여성,자영업,"['안녕, 나는 20대 남성이야.', '나도 아직 대학생이야. 우리 아버지가 운수업...","['안녕, 나는 20대 여성이야.', '나는 대학생이야. 너는 무슨 일을 해?', ...",['Bot:정말 열심히 하네. 나는 야구를 좋아해서 야구 동아리에 들어가고 싶어.'...,"그러게, 난 이만 카페가서 공부하러 가볼게. 나중에 대화하자."
4,K2-10919-CL12469-CP11587-17-05-S2.json,건강,2,"['나는 여자이다.', '나는 10대이다.', '내가 사는 곳은 속초이다', '나는...",20대,여성,학생,"['나는 요즘 치료를 받고 있다', '나는 건강관리를 안 해서 치료를 받고 있는 것...",20대,여성,무직,"['안녕, 나는 서울 사는 10대 여성이야.', '나는 요즘 모든 것이 귀찮아서 사...","['안녕, 나는 10대 여성이고, 속초에 살고 있어.', '같은 10대라니 반가워....",['Bot:솔직히 건강관리는 하나도 안하는 것 같아. 난 학교 하나 다니는 것도 벅...,응 지금은 완치됐어. 치료 받지 않게 목관리를 조심해.


In [5]:
eval_vl2['bot_persona'] = eval_vl2['bot_persona'].apply(lambda x: eval(x))
eval_vl2['user_persona'] = eval_vl2['user_persona'].apply(lambda x: eval(x))
eval_vl2['bot_dialog'] = eval_vl2['bot_dialog'].apply(lambda x: eval(x))
eval_vl2['user_dialog'] = eval_vl2['user_dialog'].apply(lambda x: eval(x))
eval_vl2['prev_dialog'] = eval_vl2['prev_dialog'].apply(lambda x: eval(x))

In [6]:
eval_vl2.loc[0, 'prev_dialog']

['Bot:나는 학원 수학강사로 일하고 있는데, 다음주부터 학생들이 기말고사라서 바빠져서 첼로연주를 쉬어야 해.',
 'User:학원 수학 선생님이셨구나. 나도 일렉기타를 배우고 싶은데, 밴드부 동아리에는 관심이 없고 학원을 알아보려고 하는데 학원비용이 많이 비쌀까?',
 'Bot:그렇게 비싸진 않을거야. 20만원 안에서 해결할 수 있을거야.',
 'User:그정도면 아르바이트해서 모을 수 있는 금액이네. 첼로 연주 배운지는 얼마나 됐어?',
 'Bot:3개월째 배우고 있는데, 중급단계를 배우고 있어.']

In [7]:
eval_vl2.loc[0, 'bot_persona'] # 지우 프롬프트도 이런 식으로 수정

['나는 여자이다.',
 '나는 20대이다.',
 '나는 고양이 두 마리를 키운다.',
 '나는 밴드부 동아리에 관심이 없다.',
 '나는 일식을 싫어한다.']

In [8]:
eval_vl2.loc[0, 'new_input']

'빠르게 습득한 거 아니야? 나는 고양이 케어하러 가야되서 다음에 연락할게.'

prev 5-turn + user input

In [9]:
from utils import convert_to_json
import json
from tqdm import tqdm
import traceback

def make_json_data(origin_data, data_name, model, prompt, persona, json_path="./"):
    print('## Data info ##')
    print(f"dataset name : {data_name} data length : {origin_data.shape}")
    print('## Model info ##')
    print(model)

    preprocessed_data = pd.DataFrame(columns=['user_persona', 'bot_persona', 'prev_dialog', 'response'])

    # row마다 하나씩 넣기
    for idx in tqdm(range(len(origin_data))):
        llm_chain = LLMChain(prompt=prompt, llm=model)
        try:
            preprocessed_data.loc[idx, 'bot_persona'] = origin_data.loc[idx, 'bot_persona']
            preprocessed_data.loc[idx, 'prev_dialog'] = origin_data.loc[idx, 'prev_dialog'][1:] + ["User:" + origin_data.loc[idx, 'new_input']]
            
            if persona == True:
                preprocessed_data.at[idx, 'user_persona'] = origin_data.loc[idx, 'user_persona']
                preprocessed_data.at[idx, 'response'] = llm_chain.invoke(
                                            {'bot_age' : origin_data.loc[idx, 'bot_age'], 
                                            'bot_gender' : origin_data.loc[idx, 'bot_gender'],
                                            'bot_persona' : ' '.join(origin_data.loc[idx, 'bot_persona']),
                                            'user_age' : origin_data.loc[idx, 'user_age'],
                                            'user_gender' : origin_data.loc[idx, 'user_gender'],
                                            'user_persona': ' '.join(origin_data.loc[idx, 'user_persona']),
                                            'new_input': origin_data.loc[idx, 'new_input'],
                                            'prev_dialog': '\n'.join(origin_data.loc[idx, 'prev_dialog'])
                                            })['text']
            else:
                preprocessed_data.at[idx, 'user_persona'] = ""
                preprocessed_data.at[idx, 'response'] = llm_chain.invoke(
                                            {'bot_age' : origin_data.loc[idx, 'bot_age'], 
                                            'bot_gender' : origin_data.loc[idx, 'bot_gender'],
                                            'bot_persona' : ' '.join(origin_data.loc[idx, 'bot_persona']),
                                            'user_age' : origin_data.loc[idx, 'user_age'],
                                            'user_gender' : origin_data.loc[idx, 'user_gender'],
                                            'user_persona': "",
                                            'new_input': origin_data.loc[idx, 'new_input'],
                                            'prev_dialog': '\n'.join(origin_data.loc[idx, 'prev_dialog'])
                                            })['text']
        # 끊겼을 때 예외처리
        except Exception as ex:
            print(f"error occured in {idx}")
            err_msg = traceback.format_exc()
            print("## Error : ", err_msg)
            data = convert_to_json(user_persona=preprocessed_data['user_persona'], bot_persona=preprocessed_data['bot_persona'],
                                   prev_dialog=preprocessed_data['prev_dialog'], response=preprocessed_data['response'])
            
            with open(f"{json_path}/{data_name}.json", "w", encoding="UTF-8") as json_file:
                json.dump(data, json_file, ensure_ascii=False)
            print(f'## JSON Converted in {json_path}/{data_name}.json##')
            print(data)
            return data
        
    data = convert_to_json(user_persona=preprocessed_data['user_persona'], bot_persona=preprocessed_data['bot_persona'],
                            prev_dialog=preprocessed_data['prev_dialog'], response=preprocessed_data['response'])
    
    with open(f"{json_path}/{data_name}.json", "w", encoding="UTF-8") as json_file:
        json.dump(data, json_file, ensure_ascii=False, indent=4)

    print(f'## JSON Converted in {json_path} as {data_name}.json##')
    print(data)
    
    return data

### OPEN-SOLAR-KO-10_7B

In [10]:
# Make sure the model path is correct for your system!
solar = LlamaCpp(
    model_path="./solar/OPEN-SOLAR-KO-10_7B.Q5_K_S.gguf",
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    callback_manager=callback_manager,
    temperature=0.9,
    top_p=0.95,
    top_k=30,
    max_tokens=256,
    n_ctx=2048,
    stop=["User:", "Bot:", "\n", "ᄏ"*6, "ᄒ"*6, "ㅠ"*6, "ᄋ"*6, "ㅋ"*6, "ㅎ"*6, "ㅠ"*6, "ㅇ"*6],
    model_kwargs={
        'min_p':0.7,
        'repeatition_penalty': 1.4,
        'no_repeat_ngram_size': 6,
        'repeat_last_n' : 10,
        'early_stopping': True
    },
    verbose=True,  # Verbose is required to pass to the callback manager
)

llama_model_loader: loaded meta data with 21 key-value pairs and 435 tensors from ./solar/OPEN-SOLAR-KO-10_7B.Q5_K_S.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 48
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_coun

In [11]:
solar_persona_vl2 = make_json_data(eval_vl2, 'GEVAL_solar_vl2_persona', solar, prompt, True, json_path="./korean_msc/preprocessing")

## Data info ##
dataset name : GEVAL_solar_vl2_persona data length : (54, 15)
## Model info ##
LlamaCpp
Params: {'model_path': './solar/OPEN-SOLAR-KO-10_7B.Q5_K_S.gguf', 'suffix': None, 'max_tokens': 256, 'temperature': 0.9, 'top_p': 0.95, 'logprobs': None, 'echo': False, 'stop_sequences': ['User:', 'Bot:', '\n', 'ᄏᄏᄏᄏᄏᄏ', 'ᄒᄒᄒᄒᄒᄒ', 'ㅠㅠㅠㅠㅠㅠ', 'ᄋᄋᄋᄋᄋᄋ', 'ㅋㅋㅋㅋㅋㅋ', 'ㅎㅎㅎㅎㅎㅎ', 'ㅠㅠㅠㅠㅠㅠ', 'ㅇㅇㅇㅇㅇㅇ'], 'repeat_penalty': 1.1, 'top_k': 30}


  0%|          | 0/54 [00:00<?, ?it/s]

3개월째 배우고 있는데, 중급단계를 배우고 있어.


llama_print_timings:        load time =   40091.78 ms
llama_print_timings:      sample time =       8.68 ms /    16 runs   (    0.54 ms per token,  1842.47 tokens per second)
llama_print_timings: prompt eval time =   40091.52 ms /   495 tokens (   80.99 ms per token,    12.35 tokens per second)
llama_print_timings:        eval time =   11104.07 ms /    15 runs   (  740.27 ms per token,     1.35 tokens per second)
llama_print_timings:       total time =   51340.88 ms /   510 tokens
  2%|▏         | 1/54 [00:51<45:30, 51.53s/it]Llama.generate: prefix-match hit


응, 난 어머니한테 잘 하는 편이야.

...

아하 그렇구나. 나는 전도사가 종교쪽 직업인 줄만 알았지, IT쪽이 있다는 건 몰랐네.


llama_print_timings:        load time =   40091.78 ms
llama_print_timings:      sample time =       3.69 ms /    26 runs   (    0.14 ms per token,  7049.89 tokens per second)
llama_print_timings: prompt eval time =   29413.81 ms /   409 tokens (   71.92 ms per token,    13.91 tokens per second)
llama_print_timings:        eval time =    4873.35 ms /    25 runs   (  194.93 ms per token,     5.13 tokens per second)
llama_print_timings:       total time =   34367.10 ms /   434 tokens
 98%|█████████▊| 53/54 [29:05<00:33, 33.47s/it]Llama.generate: prefix-match hit


지금 일 시작했는데 퇴근하고 연락할게.


llama_print_timings:        load time =   40091.78 ms
llama_print_timings:      sample time =       1.42 ms /    10 runs   (    0.14 ms per token,  7042.25 tokens per second)
llama_print_timings: prompt eval time =   29741.38 ms /   412 tokens (   72.19 ms per token,    13.85 tokens per second)
llama_print_timings:        eval time =    1731.81 ms /     9 runs   (  192.42 ms per token,     5.20 tokens per second)
llama_print_timings:       total time =   31505.43 ms /   421 tokens
100%|██████████| 54/54 [29:36<00:00, 32.90s/it]

## JSON Converted in ./korean_msc/preprocessing as GEVAL_solar_vl2_persona.json##
[{'response': '3개월째 배우고 있는데, 중급단계를 배우고 있어.', 'user_persona': ['나는 고양이 두 마리를 중성화 수술시키고 집으로 가는 길이다', '나는 친구랑 일식 약속이 잡혀있다'], 'bot_persona': ['나는 여자이다.', '나는 20대이다.', '나는 고양이 두 마리를 키운다.', '나는 밴드부 동아리에 관심이 없다.', '나는 일식을 싫어한다.'], 'prev_dialog': ['User:학원 수학 선생님이셨구나. 나도 일렉기타를 배우고 싶은데, 밴드부 동아리에는 관심이 없고 학원을 알아보려고 하는데 학원비용이 많이 비쌀까?', 'Bot:그렇게 비싸진 않을거야. 20만원 안에서 해결할 수 있을거야.', 'User:그정도면 아르바이트해서 모을 수 있는 금액이네. 첼로 연주 배운지는 얼마나 됐어?', 'Bot:3개월째 배우고 있는데, 중급단계를 배우고 있어.', 'User:빠르게 습득한 거 아니야? 나는 고양이 케어하러 가야되서 다음에 연락할게.']}, {'response': '응, 난 어머니한테 잘 하는 편이야.', 'user_persona': ['나의 형은 자동차 정비원이다'], 'bot_persona': ['나는 남자이다.', '나는 20대이다.', '나의 거주지는 대구이다.', '나는 워드프로세서 자격증을 준비한다.', '나는 단소 연주를 배우고 싶다.', '나는 요즘 어머니에 대해서 관심을 갖고 공부한다.'], 'prev_dialog': ['User:네 어머니께는 죄송하지만, 그건 다 옛날 말이야. 요즘 남자는 180cm를 못 넘기면 사람 취급도 못 받는다고 하더라고.', 'Bot:아 그렇군. 그런 말은 다소 너무하다는 생각도 가끔 들어.', 'User:그래서 더 이악물고 운동하는 것 같기도해.', 'Bot:화이팅이야. 어머니가 좀 다소 옛날분이셔서 그런 생각을 

### No Persona

In [12]:
# Make sure the model path is correct for your system!
solar_no_persona_vl2_data = make_json_data(eval_vl2, 'GEVAL_No_Persona_solar_vl2', solar, prompt, False, json_path="./korean_msc/preprocessing")

## Data info ##
dataset name : GEVAL_No_Persona_solar_vl2 data length : (54, 15)
## Model info ##
LlamaCpp
Params: {'model_path': './solar/OPEN-SOLAR-KO-10_7B.Q5_K_S.gguf', 'suffix': None, 'max_tokens': 256, 'temperature': 0.9, 'top_p': 0.95, 'logprobs': None, 'echo': False, 'stop_sequences': ['User:', 'Bot:', '\n', 'ᄏᄏᄏᄏᄏᄏ', 'ᄒᄒᄒᄒᄒᄒ', 'ㅠㅠㅠㅠㅠㅠ', 'ᄋᄋᄋᄋᄋᄋ', 'ㅋㅋㅋㅋㅋㅋ', 'ㅎㅎㅎㅎㅎㅎ', 'ㅠㅠㅠㅠㅠㅠ', 'ㅇㅇㅇㅇㅇㅇ'], 'repeat_penalty': 1.1, 'top_k': 30}


  0%|          | 0/54 [00:00<?, ?it/s]Llama.generate: prefix-match hit


네~ 잘 케어해주세요!


llama_print_timings:        load time =   40091.78 ms
llama_print_timings:      sample time =       1.12 ms /     8 runs   (    0.14 ms per token,  7149.24 tokens per second)
llama_print_timings: prompt eval time =   29470.39 ms /   405 tokens (   72.77 ms per token,    13.74 tokens per second)
llama_print_timings:        eval time =    1362.92 ms /     7 runs   (  194.70 ms per token,     5.14 tokens per second)
llama_print_timings:       total time =   30857.34 ms /   412 tokens
  2%|▏         | 1/54 [00:30<27:15, 30.87s/it]Llama.generate: prefix-match hit


응 우리 어머니는 항상 나를 지지해주셔. 어머니께 감사해.

...

그럼 정말 천직인가 보다. 나도 나중에 전도사를 해야겠어.


llama_print_timings:        load time =   40091.78 ms
llama_print_timings:      sample time =       2.27 ms /    16 runs   (    0.14 ms per token,  7054.67 tokens per second)
llama_print_timings: prompt eval time =   28047.49 ms /   386 tokens (   72.66 ms per token,    13.76 tokens per second)
llama_print_timings:        eval time =    2883.99 ms /    15 runs   (  192.27 ms per token,     5.20 tokens per second)
llama_print_timings:       total time =   30981.32 ms /   401 tokens
 98%|█████████▊| 53/54 [27:21<00:32, 32.15s/it]Llama.generate: prefix-match hit


내일 퇴근하고 연락할게


llama_print_timings:        load time =   40091.78 ms
llama_print_timings:      sample time =       1.01 ms /     7 runs   (    0.14 ms per token,  6923.84 tokens per second)
llama_print_timings: prompt eval time =   28699.45 ms /   398 tokens (   72.11 ms per token,    13.87 tokens per second)
llama_print_timings:        eval time =    1175.74 ms /     6 runs   (  195.96 ms per token,     5.10 tokens per second)
llama_print_timings:       total time =   29897.06 ms /   404 tokens
100%|██████████| 54/54 [27:51<00:00, 30.95s/it]

## JSON Converted in ./korean_msc/preprocessing as GEVAL_No_Persona_solar_vl2.json##
[{'response': '네~ 잘 케어해주세요!', 'user_persona': '', 'bot_persona': ['나는 여자이다.', '나는 20대이다.', '나는 고양이 두 마리를 키운다.', '나는 밴드부 동아리에 관심이 없다.', '나는 일식을 싫어한다.'], 'prev_dialog': ['User:학원 수학 선생님이셨구나. 나도 일렉기타를 배우고 싶은데, 밴드부 동아리에는 관심이 없고 학원을 알아보려고 하는데 학원비용이 많이 비쌀까?', 'Bot:그렇게 비싸진 않을거야. 20만원 안에서 해결할 수 있을거야.', 'User:그정도면 아르바이트해서 모을 수 있는 금액이네. 첼로 연주 배운지는 얼마나 됐어?', 'Bot:3개월째 배우고 있는데, 중급단계를 배우고 있어.', 'User:빠르게 습득한 거 아니야? 나는 고양이 케어하러 가야되서 다음에 연락할게.']}, {'response': '응 우리 어머니는 항상 나를 지지해주셔. 어머니께 감사해.', 'user_persona': '', 'bot_persona': ['나는 남자이다.', '나는 20대이다.', '나의 거주지는 대구이다.', '나는 워드프로세서 자격증을 준비한다.', '나는 단소 연주를 배우고 싶다.', '나는 요즘 어머니에 대해서 관심을 갖고 공부한다.'], 'prev_dialog': ['User:네 어머니께는 죄송하지만, 그건 다 옛날 말이야. 요즘 남자는 180cm를 못 넘기면 사람 취급도 못 받는다고 하더라고.', 'Bot:아 그렇군. 그런 말은 다소 너무하다는 생각도 가끔 들어.', 'User:그래서 더 이악물고 운동하는 것 같기도해.', 'Bot:화이팅이야. 어머니가 좀 다소 옛날분이셔서 그런 생각을 하셨나봐.', 'User:괜찮아. 너는 어머니하고 사이 좋아?']}, {'response': '선생님은 영어공부를 많이 해서 미국가서

### additional

In [13]:
solar_persona_vl2 = make_json_data(eval_vl2, 'GEVAL_solar_vl2_persona_2', solar, prompt, True, json_path="./korean_msc/preprocessing")

## Data info ##
dataset name : GEVAL_solar_vl2_persona_2 data length : (54, 15)
## Model info ##
LlamaCpp
Params: {'model_path': './solar/OPEN-SOLAR-KO-10_7B.Q5_K_S.gguf', 'suffix': None, 'max_tokens': 256, 'temperature': 0.9, 'top_p': 0.95, 'logprobs': None, 'echo': False, 'stop_sequences': ['User:', 'Bot:', '\n', 'ᄏᄏᄏᄏᄏᄏ', 'ᄒᄒᄒᄒᄒᄒ', 'ㅠㅠㅠㅠㅠㅠ', 'ᄋᄋᄋᄋᄋᄋ', 'ㅋㅋㅋㅋㅋㅋ', 'ㅎㅎㅎㅎㅎㅎ', 'ㅠㅠㅠㅠㅠㅠ', 'ㅇㅇㅇㅇㅇㅇ'], 'repeat_penalty': 1.1, 'top_k': 30}


  0%|          | 0/54 [00:00<?, ?it/s]Llama.generate: prefix-match hit


3개월이면 그렇게 빠르다고 할 순 없지~. 다음주에 다시 대화하자~


llama_print_timings:        load time =   40091.78 ms
llama_print_timings:      sample time =       2.51 ms /    18 runs   (    0.14 ms per token,  7174.17 tokens per second)
llama_print_timings: prompt eval time =   30968.36 ms /   428 tokens (   72.36 ms per token,    13.82 tokens per second)
llama_print_timings:        eval time =    3234.95 ms /    17 runs   (  190.29 ms per token,     5.26 tokens per second)
llama_print_timings:       total time =   34258.24 ms /   445 tokens
  2%|▏         | 1/54 [00:34<30:15, 34.26s/it]Llama.generate: prefix-match hit


네 어머니하고 사이 정말 좋아! 우리 어머니가 나한테 항상 맛있는거 사주시거든.

...

내가 전도사로 일하면 어떨까?


llama_print_timings:        load time =   40091.78 ms
llama_print_timings:      sample time =       1.44 ms /    10 runs   (    0.14 ms per token,  6968.64 tokens per second)
llama_print_timings: prompt eval time =   29533.59 ms /   409 tokens (   72.21 ms per token,    13.85 tokens per second)
llama_print_timings:        eval time =    1699.67 ms /     9 runs   (  188.85 ms per token,     5.30 tokens per second)
llama_print_timings:       total time =   31262.29 ms /   418 tokens
 98%|█████████▊| 53/54 [27:45<00:32, 32.46s/it]Llama.generate: prefix-match hit


응, 퇴근하고 연락할게.


llama_print_timings:        load time =   40091.78 ms
llama_print_timings:      sample time =       1.12 ms /     8 runs   (    0.14 ms per token,  7168.46 tokens per second)
llama_print_timings: prompt eval time =   29730.63 ms /   412 tokens (   72.16 ms per token,    13.86 tokens per second)
llama_print_timings:        eval time =    1335.94 ms /     7 runs   (  190.85 ms per token,     5.24 tokens per second)
llama_print_timings:       total time =   31089.96 ms /   419 tokens
100%|██████████| 54/54 [28:16<00:00, 31.42s/it]

## JSON Converted in ./korean_msc/preprocessing as GEVAL_solar_vl2_persona_2.json##
[{'response': '3개월이면 그렇게 빠르다고 할 순 없지~. 다음주에 다시 대화하자~', 'user_persona': ['나는 고양이 두 마리를 중성화 수술시키고 집으로 가는 길이다', '나는 친구랑 일식 약속이 잡혀있다'], 'bot_persona': ['나는 여자이다.', '나는 20대이다.', '나는 고양이 두 마리를 키운다.', '나는 밴드부 동아리에 관심이 없다.', '나는 일식을 싫어한다.'], 'prev_dialog': ['User:학원 수학 선생님이셨구나. 나도 일렉기타를 배우고 싶은데, 밴드부 동아리에는 관심이 없고 학원을 알아보려고 하는데 학원비용이 많이 비쌀까?', 'Bot:그렇게 비싸진 않을거야. 20만원 안에서 해결할 수 있을거야.', 'User:그정도면 아르바이트해서 모을 수 있는 금액이네. 첼로 연주 배운지는 얼마나 됐어?', 'Bot:3개월째 배우고 있는데, 중급단계를 배우고 있어.', 'User:빠르게 습득한 거 아니야? 나는 고양이 케어하러 가야되서 다음에 연락할게.']}, {'response': '네 어머니하고 사이 정말 좋아! 우리 어머니가 나한테 항상 맛있는거 사주시거든.', 'user_persona': ['나의 형은 자동차 정비원이다'], 'bot_persona': ['나는 남자이다.', '나는 20대이다.', '나의 거주지는 대구이다.', '나는 워드프로세서 자격증을 준비한다.', '나는 단소 연주를 배우고 싶다.', '나는 요즘 어머니에 대해서 관심을 갖고 공부한다.'], 'prev_dialog': ['User:네 어머니께는 죄송하지만, 그건 다 옛날 말이야. 요즘 남자는 180cm를 못 넘기면 사람 취급도 못 받는다고 하더라고.', 'Bot:아 그렇군. 그런 말은 다소 너무하다는 생각도 가끔 들어.', 'User:그래서 더 이악물고 운동하는 것 같기도해.', 

In [14]:
solar_no_persona_vl2_data = make_json_data(eval_vl2, 'GEVAL_No_Persona_solar_vl2_2', solar, prompt, False, json_path="./korean_msc/preprocessing")

## Data info ##
dataset name : GEVAL_No_Persona_solar_vl2_2 data length : (54, 15)
## Model info ##
LlamaCpp
Params: {'model_path': './solar/OPEN-SOLAR-KO-10_7B.Q5_K_S.gguf', 'suffix': None, 'max_tokens': 256, 'temperature': 0.9, 'top_p': 0.95, 'logprobs': None, 'echo': False, 'stop_sequences': ['User:', 'Bot:', '\n', 'ᄏᄏᄏᄏᄏᄏ', 'ᄒᄒᄒᄒᄒᄒ', 'ㅠㅠㅠㅠㅠㅠ', 'ᄋᄋᄋᄋᄋᄋ', 'ㅋㅋㅋㅋㅋㅋ', 'ㅎㅎㅎㅎㅎㅎ', 'ㅠㅠㅠㅠㅠㅠ', 'ㅇㅇㅇㅇㅇㅇ'], 'repeat_penalty': 1.1, 'top_k': 30}


  0%|          | 0/54 [00:00<?, ?it/s]Llama.generate: prefix-match hit


그래요. 다음에 또 연락해요~


llama_print_timings:        load time =   40091.78 ms
llama_print_timings:      sample time =       1.13 ms /     8 runs   (    0.14 ms per token,  7060.90 tokens per second)
llama_print_timings: prompt eval time =   29260.16 ms /   405 tokens (   72.25 ms per token,    13.84 tokens per second)
llama_print_timings:        eval time =    1353.89 ms /     7 runs   (  193.41 ms per token,     5.17 tokens per second)
llama_print_timings:       total time =   30638.86 ms /   412 tokens
  2%|▏         | 1/54 [00:30<27:04, 30.64s/it]Llama.generate: prefix-match hit


어릴 적에는 많이 싸웠는데 이제 다 크고 나니까 사이 좋더라고.

...

그럼 나도 한번 도전해볼까?


llama_print_timings:        load time =   40091.78 ms
llama_print_timings:      sample time =       1.29 ms /     9 runs   (    0.14 ms per token,  6993.01 tokens per second)
llama_print_timings: prompt eval time =   27670.90 ms /   386 tokens (   71.69 ms per token,    13.95 tokens per second)
llama_print_timings:        eval time =    1519.11 ms /     8 runs   (  189.89 ms per token,     5.27 tokens per second)
llama_print_timings:       total time =   29216.55 ms /   394 tokens
 98%|█████████▊| 53/54 [27:13<00:31, 31.45s/it]Llama.generate: prefix-match hit


안녕히 계세요!


llama_print_timings:        load time =   40091.78 ms
llama_print_timings:      sample time =       0.99 ms /     7 runs   (    0.14 ms per token,  7070.71 tokens per second)
llama_print_timings: prompt eval time =   28669.14 ms /   398 tokens (   72.03 ms per token,    13.88 tokens per second)
llama_print_timings:        eval time =    1138.51 ms /     6 runs   (  189.75 ms per token,     5.27 tokens per second)
llama_print_timings:       total time =   29828.82 ms /   404 tokens
100%|██████████| 54/54 [27:43<00:00, 30.81s/it]

## JSON Converted in ./korean_msc/preprocessing as GEVAL_No_Persona_solar_vl2_2.json##
[{'response': '그래요. 다음에 또 연락해요~', 'user_persona': '', 'bot_persona': ['나는 여자이다.', '나는 20대이다.', '나는 고양이 두 마리를 키운다.', '나는 밴드부 동아리에 관심이 없다.', '나는 일식을 싫어한다.'], 'prev_dialog': ['User:학원 수학 선생님이셨구나. 나도 일렉기타를 배우고 싶은데, 밴드부 동아리에는 관심이 없고 학원을 알아보려고 하는데 학원비용이 많이 비쌀까?', 'Bot:그렇게 비싸진 않을거야. 20만원 안에서 해결할 수 있을거야.', 'User:그정도면 아르바이트해서 모을 수 있는 금액이네. 첼로 연주 배운지는 얼마나 됐어?', 'Bot:3개월째 배우고 있는데, 중급단계를 배우고 있어.', 'User:빠르게 습득한 거 아니야? 나는 고양이 케어하러 가야되서 다음에 연락할게.']}, {'response': '어릴 적에는 많이 싸웠는데 이제 다 크고 나니까 사이 좋더라고.', 'user_persona': '', 'bot_persona': ['나는 남자이다.', '나는 20대이다.', '나의 거주지는 대구이다.', '나는 워드프로세서 자격증을 준비한다.', '나는 단소 연주를 배우고 싶다.', '나는 요즘 어머니에 대해서 관심을 갖고 공부한다.'], 'prev_dialog': ['User:네 어머니께는 죄송하지만, 그건 다 옛날 말이야. 요즘 남자는 180cm를 못 넘기면 사람 취급도 못 받는다고 하더라고.', 'Bot:아 그렇군. 그런 말은 다소 너무하다는 생각도 가끔 들어.', 'User:그래서 더 이악물고 운동하는 것 같기도해.', 'Bot:화이팅이야. 어머니가 좀 다소 옛날분이셔서 그런 생각을 하셨나봐.', 'User:괜찮아. 너는 어머니하고 사이 좋아?']}, {'response': '어학연수 가서도 가끔 

In [15]:
solar_persona_vl2 = make_json_data(eval_vl2, 'GEVAL_solar_vl2_persona_3', solar, prompt, True, json_path="./korean_msc/preprocessing")

## Data info ##
dataset name : GEVAL_solar_vl2_persona_3 data length : (54, 15)
## Model info ##
LlamaCpp
Params: {'model_path': './solar/OPEN-SOLAR-KO-10_7B.Q5_K_S.gguf', 'suffix': None, 'max_tokens': 256, 'temperature': 0.9, 'top_p': 0.95, 'logprobs': None, 'echo': False, 'stop_sequences': ['User:', 'Bot:', '\n', 'ᄏᄏᄏᄏᄏᄏ', 'ᄒᄒᄒᄒᄒᄒ', 'ㅠㅠㅠㅠㅠㅠ', 'ᄋᄋᄋᄋᄋᄋ', 'ㅋㅋㅋㅋㅋㅋ', 'ㅎㅎㅎㅎㅎㅎ', 'ㅠㅠㅠㅠㅠㅠ', 'ㅇㅇㅇㅇㅇㅇ'], 'repeat_penalty': 1.1, 'top_k': 30}


  0%|          | 0/54 [00:00<?, ?it/s]Llama.generate: prefix-match hit


고양이들은 잘 케어해줘. 다음에 또 연락하자.


llama_print_timings:        load time =   40091.78 ms
llama_print_timings:      sample time =       2.10 ms /    15 runs   (    0.14 ms per token,  7129.28 tokens per second)
llama_print_timings: prompt eval time =   30854.51 ms /   428 tokens (   72.09 ms per token,    13.87 tokens per second)
llama_print_timings:        eval time =    2661.11 ms /    14 runs   (  190.08 ms per token,     5.26 tokens per second)
llama_print_timings:       total time =   33561.15 ms /   442 tokens
  2%|▏         | 1/54 [00:33<29:39, 33.57s/it]Llama.generate: prefix-match hit


응 어머니랑 사이 좋지. 어머니가 날 많이 예뻐해주셔.

...

그렇구나. 나도 교회에 다니는데, 요즘 목사님 강의가 너무 재밌어!


llama_print_timings:        load time =   40091.78 ms
llama_print_timings:      sample time =       2.69 ms /    19 runs   (    0.14 ms per token,  7076.35 tokens per second)
llama_print_timings: prompt eval time =   29688.33 ms /   409 tokens (   72.59 ms per token,    13.78 tokens per second)
llama_print_timings:        eval time =    3497.28 ms /    18 runs   (  194.29 ms per token,     5.15 tokens per second)
llama_print_timings:       total time =   33244.00 ms /   427 tokens
 98%|█████████▊| 53/54 [27:52<00:32, 32.97s/it]Llama.generate: prefix-match hit


너도 너무 스트레스 받지마.


llama_print_timings:        load time =   40091.78 ms
llama_print_timings:      sample time =       1.15 ms /     8 runs   (    0.14 ms per token,  6962.58 tokens per second)
llama_print_timings: prompt eval time =   29708.21 ms /   412 tokens (   72.11 ms per token,    13.87 tokens per second)
llama_print_timings:        eval time =    1339.75 ms /     7 runs   (  191.39 ms per token,     5.22 tokens per second)
llama_print_timings:       total time =   31071.96 ms /   419 tokens
100%|██████████| 54/54 [28:23<00:00, 31.55s/it]

## JSON Converted in ./korean_msc/preprocessing as GEVAL_solar_vl2_persona_3.json##
[{'response': '고양이들은 잘 케어해줘. 다음에 또 연락하자.', 'user_persona': ['나는 고양이 두 마리를 중성화 수술시키고 집으로 가는 길이다', '나는 친구랑 일식 약속이 잡혀있다'], 'bot_persona': ['나는 여자이다.', '나는 20대이다.', '나는 고양이 두 마리를 키운다.', '나는 밴드부 동아리에 관심이 없다.', '나는 일식을 싫어한다.'], 'prev_dialog': ['User:학원 수학 선생님이셨구나. 나도 일렉기타를 배우고 싶은데, 밴드부 동아리에는 관심이 없고 학원을 알아보려고 하는데 학원비용이 많이 비쌀까?', 'Bot:그렇게 비싸진 않을거야. 20만원 안에서 해결할 수 있을거야.', 'User:그정도면 아르바이트해서 모을 수 있는 금액이네. 첼로 연주 배운지는 얼마나 됐어?', 'Bot:3개월째 배우고 있는데, 중급단계를 배우고 있어.', 'User:빠르게 습득한 거 아니야? 나는 고양이 케어하러 가야되서 다음에 연락할게.']}, {'response': '응 어머니랑 사이 좋지. 어머니가 날 많이 예뻐해주셔.', 'user_persona': ['나의 형은 자동차 정비원이다'], 'bot_persona': ['나는 남자이다.', '나는 20대이다.', '나의 거주지는 대구이다.', '나는 워드프로세서 자격증을 준비한다.', '나는 단소 연주를 배우고 싶다.', '나는 요즘 어머니에 대해서 관심을 갖고 공부한다.'], 'prev_dialog': ['User:네 어머니께는 죄송하지만, 그건 다 옛날 말이야. 요즘 남자는 180cm를 못 넘기면 사람 취급도 못 받는다고 하더라고.', 'Bot:아 그렇군. 그런 말은 다소 너무하다는 생각도 가끔 들어.', 'User:그래서 더 이악물고 운동하는 것 같기도해.', 'Bot:화이팅이야. 어머니가 좀 다소 옛날분이

In [16]:
solar_no_persona_vl2_data = make_json_data(eval_vl2, 'GEVAL_No_Persona_solar_vl2_3', solar, prompt, False, json_path="./korean_msc/preprocessing")

## Data info ##
dataset name : GEVAL_No_Persona_solar_vl2_3 data length : (54, 15)
## Model info ##
LlamaCpp
Params: {'model_path': './solar/OPEN-SOLAR-KO-10_7B.Q5_K_S.gguf', 'suffix': None, 'max_tokens': 256, 'temperature': 0.9, 'top_p': 0.95, 'logprobs': None, 'echo': False, 'stop_sequences': ['User:', 'Bot:', '\n', 'ᄏᄏᄏᄏᄏᄏ', 'ᄒᄒᄒᄒᄒᄒ', 'ㅠㅠㅠㅠㅠㅠ', 'ᄋᄋᄋᄋᄋᄋ', 'ㅋㅋㅋㅋㅋㅋ', 'ㅎㅎㅎㅎㅎㅎ', 'ㅠㅠㅠㅠㅠㅠ', 'ㅇㅇㅇㅇㅇㅇ'], 'repeat_penalty': 1.1, 'top_k': 30}


  0%|          | 0/54 [00:00<?, ?it/s]Llama.generate: prefix-match hit


그래요 다음에 또 이야기해요.


llama_print_timings:        load time =   40091.78 ms
llama_print_timings:      sample time =       0.98 ms /     7 runs   (    0.14 ms per token,  7135.58 tokens per second)
llama_print_timings: prompt eval time =   29368.43 ms /   405 tokens (   72.51 ms per token,    13.79 tokens per second)
llama_print_timings:        eval time =    1150.99 ms /     6 runs   (  191.83 ms per token,     5.21 tokens per second)
llama_print_timings:       total time =   30540.25 ms /   411 tokens
  2%|▏         | 1/54 [00:30<26:58, 30.54s/it]Llama.generate: prefix-match hit


어 어머니는 내가 가장 좋아하는 분이셔.

...

아 그렇구나. 그래도 많이 버는 거네?


llama_print_timings:        load time =   40091.78 ms
llama_print_timings:      sample time =       1.72 ms /    12 runs   (    0.14 ms per token,  6972.69 tokens per second)
llama_print_timings: prompt eval time =   27724.72 ms /   386 tokens (   71.83 ms per token,    13.92 tokens per second)
llama_print_timings:        eval time =    2080.45 ms /    11 runs   (  189.13 ms per token,     5.29 tokens per second)
llama_print_timings:       total time =   29841.77 ms /   397 tokens
 98%|█████████▊| 53/54 [27:04<00:31, 31.14s/it]Llama.generate: prefix-match hit


나한테 할 말 있으면 언제든지 물어봐!


llama_print_timings:        load time =   40091.78 ms
llama_print_timings:      sample time =       1.55 ms /    11 runs   (    0.14 ms per token,  7101.36 tokens per second)
llama_print_timings: prompt eval time =   28295.84 ms /   398 tokens (   71.10 ms per token,    14.07 tokens per second)
llama_print_timings:        eval time =    1905.82 ms /    10 runs   (  190.58 ms per token,     5.25 tokens per second)
llama_print_timings:       total time =   30234.40 ms /   408 tokens
100%|██████████| 54/54 [27:34<00:00, 30.64s/it]

## JSON Converted in ./korean_msc/preprocessing as GEVAL_No_Persona_solar_vl2_3.json##
[{'response': '그래요 다음에 또 이야기해요.', 'user_persona': '', 'bot_persona': ['나는 여자이다.', '나는 20대이다.', '나는 고양이 두 마리를 키운다.', '나는 밴드부 동아리에 관심이 없다.', '나는 일식을 싫어한다.'], 'prev_dialog': ['User:학원 수학 선생님이셨구나. 나도 일렉기타를 배우고 싶은데, 밴드부 동아리에는 관심이 없고 학원을 알아보려고 하는데 학원비용이 많이 비쌀까?', 'Bot:그렇게 비싸진 않을거야. 20만원 안에서 해결할 수 있을거야.', 'User:그정도면 아르바이트해서 모을 수 있는 금액이네. 첼로 연주 배운지는 얼마나 됐어?', 'Bot:3개월째 배우고 있는데, 중급단계를 배우고 있어.', 'User:빠르게 습득한 거 아니야? 나는 고양이 케어하러 가야되서 다음에 연락할게.']}, {'response': '어 어머니는 내가 가장 좋아하는 분이셔.', 'user_persona': '', 'bot_persona': ['나는 남자이다.', '나는 20대이다.', '나의 거주지는 대구이다.', '나는 워드프로세서 자격증을 준비한다.', '나는 단소 연주를 배우고 싶다.', '나는 요즘 어머니에 대해서 관심을 갖고 공부한다.'], 'prev_dialog': ['User:네 어머니께는 죄송하지만, 그건 다 옛날 말이야. 요즘 남자는 180cm를 못 넘기면 사람 취급도 못 받는다고 하더라고.', 'Bot:아 그렇군. 그런 말은 다소 너무하다는 생각도 가끔 들어.', 'User:그래서 더 이악물고 운동하는 것 같기도해.', 'Bot:화이팅이야. 어머니가 좀 다소 옛날분이셔서 그런 생각을 하셨나봐.', 'User:괜찮아. 너는 어머니하고 사이 좋아?']}, {'response': '아이고... 걱정된다... 다녀와서 또 연락줘

### threshold 0.85

In [42]:
eval_vl2 = pd.read_csv('korean_msc/preprocessing/VL_session2_geval_v6_sampled3_banmal_0.85.csv')
eval_vl2.shape

(13, 18)

In [43]:
eval_vl2.head()

,id,topic,num_sessions,bot_persona,bot_age,bot_gender,bot_occupation,user_persona,user_age,user_gender,user_occupation,bot_dialog,user_dialog,prev_dialog,new_input,bot_dialog_len,user_dialog_len,prev_dialog_len
0,K2-33237-CL21405-CP22628-07-07-S2.json,가족,2,"['나는 여자이다.', '나는 20대이다.', '나는 고양이 두 마리를 키운다.',...",20대,여성,사무직,['나는 고양이 두 마리를 중성화 수술시키고 집으로 가는 길이다'],20대,여성,기능직,"['경기도에 살고 있구나. 난 20대이고 같은 경기도에 사는 여자야.', '고양이를...","['안녕, 난 경기도에서 살고 있어. 20대 여자야.', '같은 도시 사람이네. 나...","['Bot:나는 학원 수학강사로 일하고 있는데, 다음주부터 학생들이 기말고사라서 바...",빠르게 습득한 거 아니야? 나는 고양이 케어하러 가야되서 다음에 연락할게.,6,6,5
1,K2-10919-CL12469-CP11587-17-05-S2.json,건강,2,"['나는 여자이다.', '나는 10대이다.', '내가 사는 곳은 속초이다', '나는...",20대,여성,학생,"['나는 요즘 치료를 받고 있다', '나는 건강관리를 안 해서 치료를 받고 있는 것...",20대,여성,무직,"['안녕, 나는 서울 사는 10대 여성이야.', '나는 요즘 모든 것이 귀찮아서 사...","['안녕, 나는 10대 여성이고, 속초에 살고 있어.', '같은 10대라니 반가워....",['Bot:솔직히 건강관리는 하나도 안하는 것 같아. 난 학교 하나 다니는 것도 벅...,응 지금은 완치됐어. 치료 받지 않게 목관리를 조심해.,13,13,5
2,K5-02414-CL72610-CP20868-03-09-S2.json,교통,2,"['나는 여자이다.', '나는 20대이다.', '나는 앵무새 한 마리를 키운다.',...",20대,남성,학생,['나는 힙합 음악을 좋아한다'],20대,여성,학생,"['안녕, 나는 20대 남자이고 부산에 살아.', '반려동물은 키우지 않는 것 같네...","['안녕, 나는 20대 여자야.', '안녕, 혹시 반려동물은 키워?', '나는 앵무...","['Bot:그렇군. 나는 힙합 음악 좋아하는데 힙합은 어때?', 'User:힙합은 ...",원래 노래를 부르는 것을 좋아해? 지난번에는 힙합 음악도 좋아한다고 들었는데.,9,9,5
3,K2-12979-CL21957-CP22405-05-09-S2.json,군대,2,"['나는 여자이다.', '나는 20대이다.', '나는 요즘, 매일 감사하다', '나...",20대,여성,사무직,['나는 친구가 군인이다'],20대,여성,무직,"['안녕, 방송 촬영 기사 20대 여성이야.', '좋은 계절이네. 나는 요즘 매일 ...","['안녕, 나는 20대 여성 대학생이야.', '어떤 일에 감사하고 있어?', '그렇...","['Bot:벌써 하루가 지났네. 뭐하고 있어?', 'User:오늘 친구를 만나고 왔...","응, 군인 친구를 면회 가 본 적 있어?",11,9,5
4,K2-03595-CL21596-CP20720-19-05-S2.json,방송/연예,2,"['나는 여자이다.', '나는 20대이다.', '나는 토끼 두 마리를 키운다.', ...",20대,여성,학생,['나는 @브랜드명1@를 본다'],20대,남성,학생,"['안녕, 20대 여자야.', '안녕. 무슨 일을 하는 중이야?', '나도 따야 하...","['안녕, 나도 20대 남자야. 안녕.', '나는 최근에 오픽을 따서 지금은 잠깐 ...","['Bot:오, 그렇구나. 너무 비싸지 않아?', 'User:몇년 전에 맥그리거와 ...",그리고 @브랜드명1@ 이런데서 만화도 봐. 만화를 좋아해?,12,11,5


In [44]:
eval_vl2['bot_persona'] = eval_vl2['bot_persona'].apply(lambda x: eval(x))
eval_vl2['user_persona'] = eval_vl2['user_persona'].apply(lambda x: eval(x))
eval_vl2['bot_dialog'] = eval_vl2['bot_dialog'].apply(lambda x: eval(x))
eval_vl2['user_dialog'] = eval_vl2['user_dialog'].apply(lambda x: eval(x))
eval_vl2['prev_dialog'] = eval_vl2.apply(lambda x: eval(x['prev_dialog']), axis=1)

In [45]:
eval_vl2.loc[0, 'prev_dialog']

['Bot:나는 학원 수학강사로 일하고 있는데, 다음주부터 학생들이 기말고사라서 바빠져서 첼로연주를 쉬어야 해.',
 'User:학원 수학 선생님이셨구나. 나도 일렉기타를 배우고 싶은데, 밴드부 동아리에는 관심이 없고 학원을 알아보려고 하는데 학원비용이 많이 비쌀까?',
 'Bot:그렇게 비싸진 않을거야. 20만원 안에서 해결할 수 있을거야.',
 'User:그정도면 아르바이트해서 모을 수 있는 금액이네. 첼로 연주 배운지는 얼마나 됐어?',
 'Bot:3개월째 배우고 있는데, 중급단계를 배우고 있어.']

In [46]:
solar_persona_vl2 = make_json_data(eval_vl2, 'GEVAL_solar_vl2_1_0.85', solar, prompt, True, json_path="./korean_msc/preprocessing")

## Data info ##
dataset name : GEVAL_solar_vl2_1_0.85 data length : (13, 18)
## Model info ##
LlamaCpp
Params: {'model_path': './solar/OPEN-SOLAR-KO-10_7B.Q5_K_S.gguf', 'suffix': None, 'max_tokens': 256, 'temperature': 0.9, 'top_p': 0.95, 'logprobs': None, 'echo': False, 'stop_sequences': ['User:', 'Bot:', '\n', 'ᄏᄏᄏᄏᄏᄏ', 'ᄒᄒᄒᄒᄒᄒ', 'ㅠㅠㅠㅠㅠㅠ', 'ᄋᄋᄋᄋᄋᄋ', 'ㅋㅋㅋㅋㅋㅋ', 'ㅎㅎㅎㅎㅎㅎ', 'ㅠㅠㅠㅠㅠㅠ', 'ㅇㅇㅇㅇㅇㅇ'], 'repeat_penalty': 1.1, 'top_k': 30}


  0%|          | 0/13 [00:00<?, ?it/s]Llama.generate: prefix-match hit


그럼, 안녕히 가세요!


llama_print_timings:        load time =   40091.78 ms
llama_print_timings:      sample time =       5.22 ms /     8 runs   (    0.65 ms per token,  1532.86 tokens per second)
llama_print_timings: prompt eval time =   36207.01 ms /   419 tokens (   86.41 ms per token,    11.57 tokens per second)
llama_print_timings:        eval time =   11564.70 ms /     7 runs   ( 1652.10 ms per token,     0.61 tokens per second)
llama_print_timings:       total time =   48057.66 ms /   426 tokens
  8%|▊         | 1/13 [00:48<09:37, 48.10s/it]Llama.generate: prefix-match hit


고마워. 목 관리 열심히 해야겠다.

...

근데 패션쇼는 보통 몇시 때 시작하는 거야?


llama_print_timings:        load time =   40091.78 ms
llama_print_timings:      sample time =       2.10 ms /    13 runs   (    0.16 ms per token,  6205.25 tokens per second)
llama_print_timings: prompt eval time =   36528.75 ms /   464 tokens (   78.73 ms per token,    12.70 tokens per second)
llama_print_timings:        eval time =    2577.64 ms /    12 runs   (  214.80 ms per token,     4.66 tokens per second)
llama_print_timings:       total time =   39165.61 ms /   476 tokens
 92%|█████████▏| 12/13 [06:57<00:34, 34.72s/it]Llama.generate: prefix-match hit


이 노래는 팝송이지!


llama_print_timings:        load time =   40091.78 ms
llama_print_timings:      sample time =       1.13 ms /     8 runs   (    0.14 ms per token,  7060.90 tokens per second)
llama_print_timings: prompt eval time =   29779.16 ms /   394 tokens (   75.58 ms per token,    13.23 tokens per second)
llama_print_timings:        eval time =    1442.60 ms /     7 runs   (  206.09 ms per token,     4.85 tokens per second)
llama_print_timings:       total time =   31248.57 ms /   401 tokens
100%|██████████| 13/13 [07:29<00:00, 34.56s/it]

## JSON Converted in ./korean_msc/preprocessing as GEVAL_solar_vl2_1_0.85.json##
[{'response': '그럼, 안녕히 가세요!', 'user_persona': ['나는 고양이 두 마리를 중성화 수술시키고 집으로 가는 길이다'], 'bot_persona': ['나는 여자이다.', '나는 20대이다.', '나는 고양이 두 마리를 키운다.', '나는 밴드부 동아리에 관심이 없다.', '나는 일식을 싫어한다.'], 'prev_dialog': ['User:학원 수학 선생님이셨구나. 나도 일렉기타를 배우고 싶은데, 밴드부 동아리에는 관심이 없고 학원을 알아보려고 하는데 학원비용이 많이 비쌀까?', 'Bot:그렇게 비싸진 않을거야. 20만원 안에서 해결할 수 있을거야.', 'User:그정도면 아르바이트해서 모을 수 있는 금액이네. 첼로 연주 배운지는 얼마나 됐어?', 'Bot:3개월째 배우고 있는데, 중급단계를 배우고 있어.', 'User:빠르게 습득한 거 아니야? 나는 고양이 케어하러 가야되서 다음에 연락할게.']}, {'response': '고마워. 목 관리 열심히 해야겠다.', 'user_persona': ['나는 요즘 치료를 받고 있다', '나는 건강관리를 안 해서 치료를 받고 있는 것 같다', '나는 장이 안 좋아서 병원에 다니며 치료를 받고 있다'], 'bot_persona': ['나는 여자이다.', '나는 10대이다.', '내가 사는 곳은 속초이다', '나는 반려동물 미용사가 되고 싶다', '나는 동화를 가끔 읽는다', '나는 파란색을 싫어한다'], 'prev_dialog': ['User:나는 건강관리를 안해서 치료를 받고 있는 것 같아. @이름@님은 수술 같은 큰 치료를 받아본 적이 있어?', 'Bot:아니, 아직 수술은 안해봤어. 나중에 고의로든 타의로든 하게 될지도 모르는데 마취가 너무 무서운 것 같아.', 'User:나는 편도절제술을 해봤어. 아주 아픈 치료였지. @이름@ 너는 편도염에

In [47]:
solar_no_persona_vl2_data = make_json_data(eval_vl2, 'GEVAL_No_Persona_solar_vl2_1_0.85', solar, prompt, False, json_path="./korean_msc/preprocessing")

## Data info ##
dataset name : GEVAL_No_Persona_solar_vl2_1_0.85 data length : (13, 18)
## Model info ##
LlamaCpp
Params: {'model_path': './solar/OPEN-SOLAR-KO-10_7B.Q5_K_S.gguf', 'suffix': None, 'max_tokens': 256, 'temperature': 0.9, 'top_p': 0.95, 'logprobs': None, 'echo': False, 'stop_sequences': ['User:', 'Bot:', '\n', 'ᄏᄏᄏᄏᄏᄏ', 'ᄒᄒᄒᄒᄒᄒ', 'ㅠㅠㅠㅠㅠㅠ', 'ᄋᄋᄋᄋᄋᄋ', 'ㅋㅋㅋㅋㅋㅋ', 'ㅎㅎㅎㅎㅎㅎ', 'ㅠㅠㅠㅠㅠㅠ', 'ㅇㅇㅇㅇㅇㅇ'], 'repeat_penalty': 1.1, 'top_k': 30}


  0%|          | 0/13 [00:00<?, ?it/s]Llama.generate: prefix-match hit


알겠어. 다음에 연락줘.


llama_print_timings:        load time =   40091.78 ms
llama_print_timings:      sample time =       1.12 ms /     8 runs   (    0.14 ms per token,  7111.11 tokens per second)
llama_print_timings: prompt eval time =   33183.33 ms /   422 tokens (   78.63 ms per token,    12.72 tokens per second)
llama_print_timings:        eval time =    1375.75 ms /     7 runs   (  196.54 ms per token,     5.09 tokens per second)
llama_print_timings:       total time =   34589.79 ms /   429 tokens
  8%|▊         | 1/13 [00:34<06:55, 34.60s/it]Llama.generate: prefix-match hit


그래야겠다! @이름@님은 건강관리 뭐뭐 해?

...

그렇구나. 그럼 내가 모델인 걸 알아보고 패션쇼에 대한 질문을 해주면 돼?


llama_print_timings:        load time =   40091.78 ms
llama_print_timings:      sample time =       3.24 ms /    21 runs   (    0.15 ms per token,  6487.49 tokens per second)
llama_print_timings: prompt eval time =   34295.77 ms /   452 tokens (   75.88 ms per token,    13.18 tokens per second)
llama_print_timings:        eval time =    3920.95 ms /    20 runs   (  196.05 ms per token,     5.10 tokens per second)
llama_print_timings:       total time =   38308.87 ms /   472 tokens
 92%|█████████▏| 12/13 [06:32<00:34, 34.47s/it]Llama.generate: prefix-match hit


그러시구나, 나도 이 노래가 너무 좋아서 자주 듣는 노래 중 하나야.


llama_print_timings:        load time =   40091.78 ms
llama_print_timings:      sample time =       2.61 ms /    18 runs   (    0.15 ms per token,  6893.91 tokens per second)
llama_print_timings: prompt eval time =   29266.24 ms /   388 tokens (   75.43 ms per token,    13.26 tokens per second)
llama_print_timings:        eval time =    3538.32 ms /    17 runs   (  208.14 ms per token,     4.80 tokens per second)
llama_print_timings:       total time =   32869.46 ms /   405 tokens
100%|██████████| 13/13 [07:05<00:00, 32.74s/it]

## JSON Converted in ./korean_msc/preprocessing as GEVAL_No_Persona_solar_vl2_1_0.85.json##
[{'response': '알겠어. 다음에 연락줘.', 'user_persona': '', 'bot_persona': ['나는 여자이다.', '나는 20대이다.', '나는 고양이 두 마리를 키운다.', '나는 밴드부 동아리에 관심이 없다.', '나는 일식을 싫어한다.'], 'prev_dialog': ['User:학원 수학 선생님이셨구나. 나도 일렉기타를 배우고 싶은데, 밴드부 동아리에는 관심이 없고 학원을 알아보려고 하는데 학원비용이 많이 비쌀까?', 'Bot:그렇게 비싸진 않을거야. 20만원 안에서 해결할 수 있을거야.', 'User:그정도면 아르바이트해서 모을 수 있는 금액이네. 첼로 연주 배운지는 얼마나 됐어?', 'Bot:3개월째 배우고 있는데, 중급단계를 배우고 있어.', 'User:빠르게 습득한 거 아니야? 나는 고양이 케어하러 가야되서 다음에 연락할게.']}, {'response': '그래야겠다! @이름@님은 건강관리 뭐뭐 해?', 'user_persona': '', 'bot_persona': ['나는 여자이다.', '나는 10대이다.', '내가 사는 곳은 속초이다', '나는 반려동물 미용사가 되고 싶다', '나는 동화를 가끔 읽는다', '나는 파란색을 싫어한다'], 'prev_dialog': ['User:나는 건강관리를 안해서 치료를 받고 있는 것 같아. @이름@님은 수술 같은 큰 치료를 받아본 적이 있어?', 'Bot:아니, 아직 수술은 안해봤어. 나중에 고의로든 타의로든 하게 될지도 모르는데 마취가 너무 무서운 것 같아.', 'User:나는 편도절제술을 해봤어. 아주 아픈 치료였지. @이름@ 너는 편도염에 자주 걸리는 편이야?', 'Bot:편도염 아니야. 한번도 걸려본 적이 없는데 너무 아플 것 같아. 지금은 어때?', 'User:응 지금은 완치됐어. 치료 받지 않게 목관리를 조심

### Mistral-ko-7B

In [ ]:

# Make sure the model path is correct for your system!
mistral = LlamaCpp(
    model_path="./mistral/ggml-model-q5_k_m.gguf",
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    callback_manager=callback_manager,
    temperature=0.6,
    top_p=1,
    max_tokens=128,
    stop=["Person1:", "Person2:"],
    n_ctx=4096,
    verbose=True,  # Verbose is required to pass to the callback manager
)
mistral_vl2_data = make_json_data(eval_vl2, 'mistral_vl2_temp_0.6_1', mistral, prompt, json_path="./korean_msc/preprocessing")